In [457]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random

## 读入数据

In [458]:
df = pd.read_csv('./attachment3.csv',header=None,names=range(12))
df.fillna(-1,inplace=True)
df = df.astype('int64')

In [459]:
from tqdm import tqdm

## 生成CFG

In [460]:
N = len(df) # 节点个数
M = np.zeros([N, N])
for i in tqdm(range(N)):
    temp = df.iloc[i,:].to_list()
    for j in range(N):
        if j in temp  and i != j:
            M[i,j] = 1
        else:
            M[i,j] = 0

100%|██████████| 607/607 [00:00<00:00, 1813.74it/s]


In [461]:
# plt.figure(figsize=(200, 200), dpi=80)
G = nx.from_numpy_matrix(M, create_using=nx.DiGraph)
# nx.draw(G, node_size=100,with_labels = True)
# plt.show()

In [462]:
# # test
# G = nx.DiGraph()
# G.add_edges_from([(0,1),(1,2),(2,4),(2,5),(4,6),(5,6),(1,3),(5,7)])
# N = 8

In [463]:
# nx.draw(G,with_labels = True)
# plt.show()

## 查找输入点和输出点

In [464]:
max_ = 1000
max_list = []
for item in range(N):
    frontiers = list(G.predecessors(item))
    if len(frontiers) < max_:
        max_list = []
        max_ = len(frontiers)
        max_list.append(item)
    elif len(frontiers)==max_:
        max_list.append(item)
max_list

[365]

In [465]:
max_

0

In [466]:
min_ = 1000
min_list = []
for item in range(N):
    frontiers = nx.immediate_dominators(G,item)
    if len(frontiers)<min_:
        min_list = []
        min_ = len(frontiers)
        min_list.append(item)
    elif len(frontiers)==min_:
        min_list.append(item)
min_list

[2, 88, 360, 381, 454]

In [467]:
min_

1

设置输入点为N，输出点为N+1

In [468]:
## start: N+1
for item in max_list:
    G.add_edges_from([(N+1,item)])

In [469]:
## end: N
for item in min_list:
    G.add_edges_from([(item,N)])

In [470]:
def MakeCDG(G,start,end,leng):
    G.add_edges_from([(leng,start)])
    G.add_edges_from([(end,leng+1)])
    G.add_edges_from([(leng,leng+1)])
    G_reverse = G.reverse()
    # print(G_reverse.nodes)
    # print(leng+1)
    frontiers = nx.dominance_frontiers(G_reverse,leng+1)
    # print(frontiers)
    CDG = nx.DiGraph()
    for item in frontiers:
        frontiers_set = frontiers[item]
        for subitem in frontiers_set:
            if subitem == leng+1:
                print("!")
            else:
                CDG.add_edges_from([(subitem,item)])
    return CDG

In [471]:
CDG = MakeCDG(G,N+1,N,N+2)

In [472]:
a = set(CDG.nodes())
a.remove(N+2)
a.remove(N+1)
a.remove(N)
CDG = nx.DiGraph(nx.subgraph(CDG,a))

In [473]:
len(nx.immediate_dominators(CDG,365))

607

绘图

In [474]:
# nx.draw_networkx(CDG)
# plt.show()

生成CDG关联矩阵

In [475]:
CDG_matrix = nx.to_numpy_array(CDG,nodelist=range(N))
CDG_matrix.shape

(607, 607)

In [476]:
CDG_matrix

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [477]:
# for item in range(len(CDG_matrix)):
#     if sum(CDG_matrix[item,:])>1:
#         print(item)

## 生成数据关联

In [478]:
df2 = pd.read_csv('./attachment2.csv',header=None,names=range(37))

In [479]:
df2

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,W,X0,X1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,604,R,X122,X123,X694,X604,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1210,605,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1211,605,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1212,606,W,X90,X91,X93,X96,X97,X100,X112,X356,...,X349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 读取dict

In [480]:
from collections import defaultdict

In [481]:
write_dict = defaultdict(list)
read_dict = defaultdict(list)
write_var_dict = defaultdict(list)
read_var_dict = defaultdict(list)
for i in tqdm(range(len(df2))):
    temp = df2.iloc[i,:].to_list()
    if temp[1] == 'W':
        for j in range(2,len(temp)):
            if str(temp[j]) != "nan":
                write_dict[temp[0]].append(temp[j])
                write_var_dict[temp[j]].append(temp[0])
            else:
                break
    elif temp[1] == 'R':
        for j in range(2,len(temp)):
            if str(temp[j]) != "nan":
                read_dict[temp[0]].append(temp[j])
                read_var_dict[temp[j]].append(temp[0])
            else:
                break     

100%|██████████| 1214/1214 [00:00<00:00, 5370.87it/s]


#### 写+*约束

In [482]:
write_G = nx.DiGraph()
for i in range(N):
    write_G.add_node(i)

In [483]:
for item in tqdm(write_dict):
    for subitem in write_dict[item]:
        for another_point in write_var_dict[subitem]+read_var_dict[subitem]:
            if item != another_point:
                if nx.has_path(G,item,another_point):
                    write_G.add_edges_from([(item,another_point)])

100%|██████████| 360/360 [00:02<00:00, 150.37it/s]


In [484]:
write_matrix = nx.to_numpy_array(write_G,nodelist=range(N))

In [485]:
write_matrix.shape

(607, 607)

读+写约束

In [486]:
read_G = nx.DiGraph()
for i in range(N):
    read_G.add_node(i)

In [487]:
for item in tqdm(read_dict):
    for subitem in read_dict[item]:
        for another_point in write_var_dict[subitem]:
            if item != another_point:
                if nx.has_path(G,item,another_point):
                    read_G.add_edges_from([(item,another_point)])

100%|██████████| 373/373 [00:01<00:00, 302.64it/s]


In [488]:
read_matrix = nx.to_numpy_array(read_G,nodelist=range(N))

In [489]:
read_matrix.shape

(607, 607)

### 合并约束矩阵

权重>=2为强约束，权重=1为弱约束

In [490]:
cob_constrain_matrix = np.sign(CDG_matrix+read_matrix) + write_matrix*2

In [491]:
cob_constrain_matrix

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [492]:
cob_G = nx.from_numpy_matrix(cob_constrain_matrix,parallel_edges=False, create_using=nx.DiGraph)

In [493]:
nx.shortest_path(CDG,max_list[0], min_list[4])

[365, 390, 407, 469, 456, 454]

In [494]:
len(nx.dag_longest_path(cob_G))

61

## 问题一

#### 贪心算法

In [495]:
df3 = pd.read_csv('./attachment1.csv',names=range(5))
df3.fillna(-1,inplace=True)

In [496]:
df3

,0,1,2,3,4
0,BLOCK,TCAM,HASH,ALU,QUALIFY
1,0,0,0,2,0
2,1,0,0,0,0
3,2,0,0,0,0
4,3,0,0,0,0
...,...,...,...,...,...
603,602,0,0,1,1
604,603,0,0,18,8
605,604,0,0,7,0
606,605,0,0,0,0


In [497]:
N = len(df3) 
property_dict = {}
for i in tqdm(range(1,N)):
    temp = df3.iloc[i,:].to_list()
    property_dict[int(temp[0])] = np.array(list(map(int, temp[1:])))
    

100%|██████████| 607/607 [00:00<00:00, 7090.87it/s]


In [498]:
edges = cob_G.edges(data=True)

In [499]:
edges_dict = {}
for item in edges:
    edges_dict[(item[0],item[1])] = item[2]['weight']
    

In [500]:
def check_ancestor(G,point,clist,Num=-1):
    # print(point)
    ancestor_nodes = G.predecessors(point)
    for item in ancestor_nodes:
        if item not in clist:
            return 0
    for item in ancestor_nodes:
        if clist[item]==Num and edges_dict[(item,point)]>1.5:
            # print(item,point)
            # print(edges_dict[(item,point)])
            return 1
    return 2

In [501]:
# def find_correct_nodes(G,point,clist):
#     all_nodes_list = []
#     extra_node_list = []
#     for item in G.edges(point):
#         # print(item)
#         if check_ancestor(G,item[1],clist):
#             if edges_dict[item] == 1:
#                 all_nodes_list.append(item[1])
#             else:
#                 extra_node_list.append(item[1])
#     return all_nodes_list,extra_node_list

def find_correct_nodes(G,point,clist,Num):
    all_nodes_list = []
    extra_node_list = []
    reversed_fork_node_list = []
    for item in G.edges(point):
        result = check_ancestor(G,item[1],clist,Num)
        if result==2:
            if edges_dict[item] == 1:
                all_nodes_list.append(item[1])
            else:
                extra_node_list.append(item[1])
        elif result==0:
            reversed_fork_node_list.append(item[1])
        else:
            # print('OK')
            extra_node_list.append(item[1])
    return all_nodes_list,extra_node_list,reversed_fork_node_list

In [506]:
def recursive_update(added_point,temp_property_arr,point_list,Num,cob_G):
    # print('----')
    # print(temp_property_arr)
    all_nodes_list = []
    extra_nodes_list = []
    random.shuffle(point_list)
    for item in point_list:
        if ((Num % 2) == 0 and TCM_even>=5 and property_dict[item][0]>0):
            pass
        else:
            if ((temp_property_arr - property_dict[item]) >= 0).all() == True:
                temp_property_arr = temp_property_arr - property_dict[item]
                # print(temp_property_arr)
                alloc_dict[Num].append(item)
                have_been_alloc_list[item] = Num
                temp_all_nodes_list,temp_extra_node_list,_ = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                # print(temp_all_nodes_list)
                all_nodes_list += temp_all_nodes_list
                extra_nodes_list += temp_extra_node_list
                added_point.append(item)
                # print(temp_extra_node_list)
            
    if all_nodes_list != []:
        # print(added_point)
        # print(temp_all_nodes_list)
        temp_temp_all_nodes_list,temp_temp_extra_node_list,temp_added_point,temp_property_arr = recursive_update(added_point,temp_property_arr,all_nodes_list,Num,cob_G)
        # print(temp_all_nodes_list)
        # print(temp_temp_all_nodes_list)
        extra_nodes_list += temp_temp_extra_node_list
        all_nodes_list += temp_temp_all_nodes_list
        added_point = temp_added_point

    return all_nodes_list,extra_nodes_list,added_point,temp_property_arr

In [507]:
def tree_recursive_update(added_point,temp_property_arr,point_list,Num,cob_G):
    all_nodes_list = []
    extra_nodes_list = []
    random.shuffle(point_list)
    for item in point_list:
        temp_all_nodes_list = []
        if ((Num % 2) == 0 and TCM_even>=5 and property_dict[item][0]>0):
            pass
        else:
            if ((temp_property_arr - property_dict[item]) >= 0).all() == True:
                temp_property_arr = temp_property_arr - property_dict[item]
                alloc_dict[Num].append(item)
                have_been_alloc_list[item] = Num
                added_point.append(item)
                temp_all_nodes_list,temp_extra_node_list,_ = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                all_nodes_list += temp_all_nodes_list
                extra_nodes_list += temp_extra_node_list
                
            
        if temp_all_nodes_list != []:
            # print(temp_all_nodes_list)
            temp_temp_all_nodes_list,temp_temp_extra_node_list,temp_added_point,temp_property_arr = tree_recursive_update(added_point,temp_property_arr,temp_all_nodes_list,Num,cob_G)
            extra_nodes_list += temp_temp_extra_node_list
            all_nodes_list += temp_temp_all_nodes_list
            added_point = temp_added_point
    return all_nodes_list,extra_nodes_list,added_point,temp_property_arr
    

In [508]:
acc_limit_arr = np.zeros((1000,4))
point_list = [start_point]
Num = 0  ## 层级
TCM_even = 0  ## 偶数数量
alloc_dict = defaultdict(list)
have_been_alloc_list = {}
TCM_even_list = []

In [510]:
mini_ = 100
for i in tqdm(range(100)):
    acc_limit_arr = np.zeros((1000,4))
    point_list = [start_point]
    Num = 0  ## 层级
    TCM_even = 0  ## 偶数数量
    alloc_dict = defaultdict(list)
    have_been_alloc_list = {}
    TCM_even_list = []
    while True:
        extra_point_list = []
        if  16 <= Num and Num<= 31:
            temp_property_arr =np.minimum(np.array([1,3,56+acc_limit_arr[Num-16,2],64+acc_limit_arr[Num-16,3]]) - acc_limit_arr[Num-16,:],np.array([1,2,56,64]))
            orig_arr = temp_property_arr
        else:
            temp_property_arr = np.array([1,2,56,64])
            orig_arr = temp_property_arr
        # print(temp_property_arr)
        # print(point_list)
        # print(have_been_alloc_list)
        temp_all_nodes_list,temp_extra_node_list,added_point,temp_property_arr = recursive_update([],temp_property_arr,point_list,Num,cob_G)
        point_list =  temp_all_nodes_list + temp_extra_node_list + point_list
        # print(point_list)
        for item in added_point:
            point_list.remove(item)
        acc_limit_arr[Num,:] =  orig_arr-temp_property_arr
        
        
        if (Num % 2) == 0 and temp_property_arr[0]== 0 and Num-16 not in TCM_even_list:
            TCM_even += 1
            TCM_even_list.append(Num)
        # print(f'{Num} 级共有 {len(alloc_dict[Num])} 个block')
        Num += 1
        
        if point_list == []:
            break
    if Num<mini_:
        mini_ = Num
        print(mini_)
        df4 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in alloc_dict.items() ]))
        df4 = df4.transpose()
        # df4.drop(index=0)
        df4.to_csv('first_problem_w_greedy.csv', header=False)
        pd.DataFrame(acc_limit_arr).to_csv("first_problem_w_greedy_alloc.csv", header=False)
    # break

  6%|▌         | 6/100 [00:00<00:03, 28.38it/s]

58
54


100%|██████████| 100/100 [00:02<00:00, 44.84it/s]


In [511]:
df4 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in alloc_dict.items() ]))
df4 = df4.transpose()
# df4.drop(index=0)
df4.to_csv('first_problem_w_greedy_tree.csv', header=False)

In [512]:
pd.DataFrame(acc_limit_arr).to_csv("first_problem_w_greedy_tree_alloc.csv", header=False)

## 问题二

In [513]:
from copy import deepcopy
def multi_head_tree_recursive_update(added_point,property_arr,point_list,Num,cob_G,reversed_fork_node_dict):
    # specail_nodes_list = []
    all_nodes_list = []
    extra_nodes_list = []
    temp_property_arr = deepcopy(property_arr)
    random.shuffle(point_list)
    for item in point_list:

        temp_all_nodes_list = []

        temp_property_arr[1] = property_arr[1]
        temp_property_arr[2] = property_arr[2]
        if ((Num % 2) == 0 and TCM_even>=5 and property_dict[item][0]>0):
            pass
        else:
            if ((temp_property_arr - property_dict[item]) >= 0).all() == True:
                
                temp_property_arr = temp_property_arr - property_dict[item]
                if item in reversed_fork_node_dict:
                    picked_out_property_arr = reversed_fork_node_dict[item]
                    new_property_arr = picked_out_property_arr - property_dict[item]
                    if (new_property_arr>=0).all() == True:
                        temp_property_arr = np.minimum(temp_property_arr,new_property_arr)
                        alloc_dict[Num].append(item)
                        have_been_alloc_list[item] = Num
                        added_point.append(item)
                        temp_all_nodes_list,temp_extra_node_list,temp_reversed_fork_node_list = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                        all_nodes_list += temp_all_nodes_list
                        extra_nodes_list += temp_extra_node_list
                        for subitem in temp_reversed_fork_node_list:
                            if subitem in reversed_fork_node_dict:
                                reversed_fork_node_dict[subitem] = np.minimum(temp_property_arr,reversed_fork_node_dict[subitem])  
                            else:
                                reversed_fork_node_dict[subitem] = temp_property_arr
                        
                    else:
                        temp_property_arr = temp_property_arr + property_dict[item]
                        
                else:
                    # print(3)
                    alloc_dict[Num].append(item)
                    have_been_alloc_list[item] = Num
                    added_point.append(item)
                    temp_all_nodes_list,temp_extra_node_list,temp_reversed_fork_node_list = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                    all_nodes_list += temp_all_nodes_list
                    extra_nodes_list += temp_extra_node_list
                    for subitem in temp_reversed_fork_node_list:
                        if subitem in reversed_fork_node_dict:
                                reversed_fork_node_dict[subitem] = np.minimum(temp_property_arr,reversed_fork_node_dict[subitem])  
                        else:
                            reversed_fork_node_dict[subitem] = temp_property_arr
 
        if temp_all_nodes_list != []:
            temp_temp_all_nodes_list,temp_temp_extra_node_list,temp_added_point,temp_property_arr,reversed_fork_node_dict = multi_head_tree_recursive_update(added_point,temp_property_arr,temp_all_nodes_list,Num,cob_G,reversed_fork_node_dict)
            extra_nodes_list += temp_temp_extra_node_list
            all_nodes_list += temp_temp_all_nodes_list
            added_point = temp_added_point

    return all_nodes_list,extra_nodes_list,added_point,temp_property_arr,reversed_fork_node_dict
    

In [514]:
acc_limit_arr = np.zeros((1000,4))
point_list = [start_point]
Num = 0  ## 层级
TCM_even = 0  ## 偶数数量
alloc_dict = defaultdict(list)
have_been_alloc_list = {}
TCM_even_list = []

In [516]:
mini_ = 100
for i in tqdm(range(50)):
    acc_limit_arr = np.zeros((10000,4))
    point_list = [start_point]
    Num = 0  ## 层级
    TCM_even = 0  ## 偶数数量
    alloc_dict = defaultdict(list)
    have_been_alloc_list = {}
    TCM_even_list = []
    all_len = 0
    while True:
        extra_point_list = []
        if  16 <= Num and Num<= 31:
            temp_property_arr =np.minimum(np.array([1,3,56+acc_limit_arr[Num-16,2],64+acc_limit_arr[Num-16,3]]) - acc_limit_arr[Num-16,:],np.array([1,2,56,64]))
            orig_arr = temp_property_arr
        else:
            temp_property_arr = np.array([1,2,56,64])
            orig_arr = temp_property_arr
        # if Num==29:
        #     print(temp_property_arr)
        # print(temp_property_arr)
        # print(point_list)
        # print(have_been_alloc_list)
        temp_all_nodes_list,temp_extra_node_list,added_point,temp_property_arr,reversed_fork_node_dict = multi_head_tree_recursive_update([],temp_property_arr,point_list,Num,cob_G,{})
        # print(point_list)
        # print(temp_all_nodes_list)
        point_list =  temp_all_nodes_list + temp_extra_node_list + point_list
        # print(reversed_fork_node_dict)
        # print(point_list)
        for item in added_point:
            point_list.remove(item)
        acc_limit_arr[Num,:] =  orig_arr-temp_property_arr
        
        
        if (Num % 2) == 0 and temp_property_arr[0]== 0 and Num-16 not in TCM_even_list:
            TCM_even += 1
            TCM_even_list.append(Num)
        # print(f'{Num} 级共有 {len(alloc_dict[Num])} 个block')
        all_len = all_len + len(alloc_dict[Num])
        Num += 1
        
        if point_list == []:
            break
        # if Num==29:
        #     break
    if Num<mini_:
        mini_ = Num
        print(mini_)
        df5 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in alloc_dict.items() ]))
        df5 = df5.transpose()
        # df4.drop(index=0)
        df5.to_csv('second_problem_w_greedy_tree.csv', header=False)
        pd.DataFrame(acc_limit_arr).to_csv("second_problem_w_greedy_alloc.csv", header=False)
# break

  6%|▌         | 3/50 [00:00<00:03, 12.76it/s]

40
38


100%|██████████| 50/50 [00:01<00:00, 32.47it/s]


In [517]:
df5 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in alloc_dict.items() ]))
df5 = df5.transpose()
# df4.drop(index=0)
df5.to_csv('second_problem_w_greedy_tree.csv', header=False)

In [518]:
pd.DataFrame(acc_limit_arr).to_csv("second_problem_w_greedy_tree_alloc.csv", header=False)

#### 权值搜索 

从后向前计算各个节点的property：（之后节点数，TCAM，HASH，ALU，QUALIFY）

In [519]:
bp_property_dict = {}

for item in tqdm(range(N-1)):
    all_points = nx.descendants(cob_G,item)
    cal_property = deepcopy(property_dict[item])
    for subitem in all_points:
        cal_property += deepcopy(property_dict[subitem])
    bp_property_dict[item] = [len(all_points)+1,cal_property]

100%|██████████| 607/607 [00:00<00:00, 1467.48it/s]


In [526]:
def recursive_update_w_right(added_point,temp_property_arr,point_list,Num,cob_G):
    # print('----')
    # print(temp_property_arr)
    all_nodes_list = []
    extra_nodes_list = []
    new_dict_list = {}
    for item in point_list:
        new_dict_list[item] = property_dict[item]
    point_list = dict(sorted(new_dict_list.items(), key=lambda item: item[1][1], reverse=True)).keys()
    
    for item in point_list:
        if ((Num % 2) == 0 and TCM_even>=5 and property_dict[item][0]>0):
            pass
        else:
            if ((temp_property_arr - property_dict[item]) >= 0).all() == True:
                temp_property_arr = temp_property_arr - property_dict[item]
                # print(temp_property_arr)
                alloc_dict[Num].append(item)
                have_been_alloc_list[item] = Num
                temp_all_nodes_list,temp_extra_node_list,_ = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                # print(temp_all_nodes_list)
                all_nodes_list += temp_all_nodes_list
                extra_nodes_list += temp_extra_node_list
                added_point.append(item)
                # print(temp_extra_node_list)
            
    if all_nodes_list != []:
        # print(added_point)
        # print(temp_all_nodes_list)
        temp_temp_all_nodes_list,temp_temp_extra_node_list,temp_added_point,temp_property_arr = recursive_update_w_right(added_point,temp_property_arr,all_nodes_list,Num,cob_G)
        # print(temp_all_nodes_list)
        # print(temp_temp_all_nodes_list)
        extra_nodes_list += temp_temp_extra_node_list
        all_nodes_list += temp_temp_all_nodes_list
        added_point = temp_added_point

    return all_nodes_list,extra_nodes_list,added_point,temp_property_arr

In [527]:
# mini_ = 100
# for i in tqdm(range(100)):
acc_limit_arr = np.zeros((1000,4))
point_list = [start_point]
Num = 0  ## 层级
TCM_even = 0  ## 偶数数量
alloc_dict = defaultdict(list)
have_been_alloc_list = {}
TCM_even_list = []
while True:
    extra_point_list = []
    if  16 <= Num and Num<= 31:
        temp_property_arr =np.minimum(np.array([1,3,56+acc_limit_arr[Num-16,2],64+acc_limit_arr[Num-16,3]]) - acc_limit_arr[Num-16,:],np.array([1,2,56,64]))
        orig_arr = temp_property_arr
    else:
        temp_property_arr = np.array([1,2,56,64])
        orig_arr = temp_property_arr
    # print(temp_property_arr)
    # print(point_list)
    # print(have_been_alloc_list)
    temp_all_nodes_list,temp_extra_node_list,added_point,temp_property_arr = recursive_update_w_right([],temp_property_arr,point_list,Num,cob_G)
    point_list =  temp_all_nodes_list + temp_extra_node_list + point_list
    # print(point_list)
    for item in added_point:
        point_list.remove(item)
    acc_limit_arr[Num,:] =  orig_arr-temp_property_arr
    
    
    if (Num % 2) == 0 and temp_property_arr[0]== 0 and Num-16 not in TCM_even_list:
        TCM_even += 1
        TCM_even_list.append(Num)
    # print(f'{Num} 级共有 {len(alloc_dict[Num])} 个block')
    Num += 1
    
    if point_list == []:
        break
# if Num<mini_:
#     mini_ = Num
    # print(mini_)
df4 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in alloc_dict.items() ]))
df4 = df4.transpose()
# df4.drop(index=0)
df4.to_csv('first_problem_w_greedy_w_right.csv', header=False)
pd.DataFrame(acc_limit_arr).to_csv("first_problem_w_greedy_alloc_w_right.csv", header=False)
    # break
Num

56

## 限制资源算法

In [529]:
def recursive_update_w_forbid_prior(added_point,temp_property_arr,point_list,Num,cob_G):
    all_nodes_list = []
    extra_nodes_list = []
    problem_report = []
    # random.shuffle(point_list)
    for item in point_list:
        if ((Num % 2) == 0 and TCM_even>=5 and property_dict[item][0]>0):
            pass
        else:
            if ((temp_property_arr - property_dict[item]) >= 0).all() == True:
                temp_property_arr = temp_property_arr - property_dict[item]
                # print(temp_property_arr)
                alloc_dict[Num].append(item)
                have_been_alloc_list[item] = Num
                temp_all_nodes_list,temp_extra_node_list,_ = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                # print(temp_all_nodes_list)
                all_nodes_list += temp_all_nodes_list
                extra_nodes_list += temp_extra_node_list
                added_point.append(item)
                # print(temp_extra_node_list)
            else:
                problem_report = [item,((temp_property_arr - property_dict[item]))]
            
    if all_nodes_list != []:
        # print(all_nodes_list)
        # print(added_point)
        # print(temp_all_nodes_list)
        temp_temp_all_nodes_list,temp_temp_extra_node_list,temp_added_point,temp_property_arr,problem_report = recursive_update_w_forbid_prior(added_point,temp_property_arr,all_nodes_list,Num,cob_G)
        # print(temp_all_nodes_list)
        # print(temp_temp_all_nodes_list)
        extra_nodes_list += temp_temp_extra_node_list
        all_nodes_list += temp_temp_all_nodes_list
        added_point = temp_added_point

    return all_nodes_list,extra_nodes_list,added_point,temp_property_arr,problem_report

In [531]:
mini_ = 100
prperty_alloc_arr = np.repeat([[1,2,56,64]],16,axis=0)
for i in tqdm(range(100)):
    acc_limit_arr = np.zeros((1000,4))
    point_list = [start_point]
    Num = 0  ## 层级
    TCM_even = 0  ## 偶数数量
    alloc_dict = defaultdict(list)
    have_been_alloc_list = {}
    TCM_even_list = []
    while True:
        extra_point_list = []
        if Num<16:
            temp_property_arr = prperty_alloc_arr[Num,:]
        elif  16 <= Num and Num<= 31:
            temp_property_arr =np.array([1,3,56+acc_limit_arr[Num-16,2],64+acc_limit_arr[Num-16,3]]) - acc_limit_arr[Num-16,:]
            orig_arr = temp_property_arr
        else:
            temp_property_arr = np.array([1,2,56,64])
            orig_arr = temp_property_arr
        temp_all_nodes_list,temp_extra_node_list,added_point,temp_property_arr,report = recursive_update_w_forbid_prior([],temp_property_arr,point_list,Num,cob_G)
        point_list =  temp_all_nodes_list + temp_extra_node_list + point_list

        for item in added_point:
            point_list.remove(item)
        acc_limit_arr[Num,:] =  orig_arr-temp_property_arr
        
        
        if (Num % 2) == 0 and temp_property_arr[0]== 0 and Num-16 not in TCM_even_list:
            TCM_even += 1
            TCM_even_list.append(Num)
        # print(f'{Num} 级共有 {len(alloc_dict[Num])} 个block')
      
        
        if 20 <= Num and Num<= 31 and len(alloc_dict[Num])<6:
            if report != []:
                ran_num = random.random()
                if ran_num<1/2:
                    prperty_alloc_arr[Num-16] =np.maximum(np.minimum(np.array([1,3,56,64]) - acc_limit_arr[Num,:]+report[1],np.array([1,2,56,64])),np.array([1,1,0,0]))
                else:
                    prperty_alloc_arr[Num-16] = np.array([1,2,56,64])
            # else:
            #     prperty_alloc_arr[Num-16] =np.maximum(np.minimum(np.array([1,3,56,64]) - acc_limit_arr[Num,:],np.array([1,1,28,32])),np.zeros(4))
        elif 16 <= Num and Num < 20:
            prperty_alloc_arr[Num-16] = np.array([1,2,56,64])
        Num += 1
        if point_list == []:
            break
    # print(Num)
    if Num<mini_:
        mini_ = Num
        print(mini_)
        df4 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in alloc_dict.items() ]))
        df4 = df4.transpose()
        # df4.drop(index=0)
        df4.to_csv('first_problem_w_greedy_limit_resource.csv', header=False)
        pd.DataFrame(acc_limit_arr).to_csv("first_problem_w_greedy_limit_resource.csv", header=False)
    # break

  9%|▉         | 9/100 [00:00<00:02, 42.63it/s]

56


100%|██████████| 100/100 [00:01<00:00, 56.28it/s]


## 深度广度搜索

In [532]:
from random import randrange


def cross_recursive_update(added_point,temp_property_arr,point_list,Num,cob_G,rnd):
    all_nodes_list = []
    extra_nodes_list = []
    # new_dict_list = {}
    # for item in point_list:
    #     new_dict_list[item] = property_dict[item]
    # point_list = dict(sorted(new_dict_list.items(), key=lambda item: item[1][0], reverse=True)).keys()
    random.shuffle(point_list)
    if rnd==0:
        # print(1)
        for item in point_list:
            temp_all_nodes_list = []
            if ((Num % 2) == 0 and TCM_even>=5 and property_dict[item][0]>0):
                pass
            else:
                if ((temp_property_arr - property_dict[item]) >= 0).all() == True:
                    temp_property_arr = temp_property_arr - property_dict[item]
                    alloc_dict[Num].append(item)
                    have_been_alloc_list[item] = Num
                    added_point.append(item)
                    temp_all_nodes_list,temp_extra_node_list,_ = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                    all_nodes_list += temp_all_nodes_list
                    extra_nodes_list += temp_extra_node_list
                    
                
            if temp_all_nodes_list != []:
                # print(temp_all_nodes_list)
                rand_num = random.random()
                if bp_property_dict[item][0]/((N-1-len(alloc_dict))/len(point_list))>rand_num:
                    rnd = 0
                else:
                    rnd = 1
                temp_temp_all_nodes_list,temp_temp_extra_node_list,temp_added_point,temp_property_arr = cross_recursive_update(added_point,temp_property_arr,temp_all_nodes_list,Num,cob_G,rnd)
                extra_nodes_list += temp_temp_extra_node_list
                all_nodes_list += temp_temp_all_nodes_list
                added_point = temp_added_point
        # return all_nodes_list,extra_nodes_list,added_point,temp_property_arr
    
    else:
        # print(2)
        for item in point_list:
            if ((Num % 2) == 0 and TCM_even>=5 and property_dict[item][0]>0):
                pass
            else:
                if ((temp_property_arr - property_dict[item]) >= 0).all() == True:
                    temp_property_arr = temp_property_arr - property_dict[item]
                    # print(temp_property_arr)
                    alloc_dict[Num].append(item)
                    have_been_alloc_list[item] = Num
                    temp_all_nodes_list,temp_extra_node_list,_ = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                    # print(temp_all_nodes_list)
                    all_nodes_list += temp_all_nodes_list
                    extra_nodes_list += temp_extra_node_list
                    added_point.append(item)
                    # print(temp_extra_node_list)
                
        if all_nodes_list != []:
            rand_num = random.random()
            # print(bp_property_dict[item][0]/((N-1-len(alloc_dict))/len(point_list)))
            # print(rand_num)
            if bp_property_dict[item][0]/((N-1-len(alloc_dict))/len(point_list))>rand_num:
                rnd = 0
            else:
                rnd = 1
            # print(added_point)
            # print(temp_all_nodes_list)
            temp_temp_all_nodes_list,temp_temp_extra_node_list,temp_added_point,temp_property_arr = cross_recursive_update(added_point,temp_property_arr,all_nodes_list,Num,cob_G,randrange)
            # print(temp_all_nodes_list)
            # print(temp_temp_all_nodes_list)
            extra_nodes_list += temp_temp_extra_node_list
            all_nodes_list += temp_temp_all_nodes_list
            added_point = temp_added_point

    return all_nodes_list,extra_nodes_list,added_point,temp_property_arr
        

In [533]:
mini_ = 100
Num_list = []
for i in tqdm(range(50)):
    acc_limit_arr = np.zeros((1000,4))
    point_list = [start_point]
    Num = 0  ## 层级
    TCM_even = 0  ## 偶数数量
    alloc_dict = defaultdict(list)
    have_been_alloc_list = {}
    TCM_even_list = []
    while True:
        extra_point_list = []
        if  16 <= Num and Num<= 31:
            temp_property_arr =np.minimum(np.array([1,3,56+acc_limit_arr[Num-16,2],64+acc_limit_arr[Num-16,3]]) - acc_limit_arr[Num-16,:],np.array([1,2,56,64]))
            orig_arr = temp_property_arr
        else:
            temp_property_arr = np.array([1,2,56,64])
            orig_arr = temp_property_arr
        # print(temp_property_arr)
        # print(point_list)
        # print(have_been_alloc_list)
        temp_all_nodes_list,temp_extra_node_list,added_point,temp_property_arr = cross_recursive_update([],temp_property_arr,point_list,Num,cob_G,0)
        point_list =  temp_all_nodes_list + temp_extra_node_list + point_list
        # print(point_list)
        for item in added_point:
            point_list.remove(item)
        acc_limit_arr[Num,:] =  orig_arr-temp_property_arr
        
        
        if (Num % 2) == 0 and temp_property_arr[0]== 0 and Num-16 not in TCM_even_list:
            TCM_even += 1
            TCM_even_list.append(Num)
        # print(f'{Num} 级共有 {len(alloc_dict[Num])} 个block')
        Num += 1
        
        if point_list == []:
            break
    Num_list.append(Num)
    if Num<mini_:
        mini_ = Num
        print(mini_)
        df4 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in alloc_dict.items() ]))
        df4 = df4.transpose()
        # df4.drop(index=0)
        df4.to_csv('first_problem_w_greedy_random.csv', header=False)
        pd.DataFrame(acc_limit_arr).to_csv("first_problem_w_greedy_alloc_random.csv", header=False)
    # break

  1%|          | 3/500 [00:00<00:18, 27.32it/s]

56
54


100%|██████████| 500/500 [00:11<00:00, 44.40it/s]


---

## 单层DP

In [167]:
from copy import deepcopy
def DP_head_tree_recursive_update(property_arr,point_list,Num,cob_G,reversed_fork_node_dict,reversed_fork_list_dict = defaultdict(set),cur_layer_list=defaultdict(set)):
    # specail_nodes_list = []
    all_nodes_list = []
    extra_nodes_list = []
    temp_property_arr = deepcopy(property_arr)
    random.shuffle(point_list)
    for item in point_list:

        temp_all_nodes_list = []

        ## 进行第二支树形搜索时重置资源数
        temp_property_arr = property_arr
        
        if ((Num % 2) == 0 and TCM_even>=5 and property_dict[item][0]>0):
            pass
        else:
            if ((temp_property_arr - property_dict[item]) >= 0).all() == True:
                
                temp_property_arr = temp_property_arr - property_dict[item]
                if item in reversed_fork_node_dict:
                    ## 汇合节点
                    picked_out_property_arr = reversed_fork_node_dict[item]
                    new_property_arr = picked_out_property_arr - property_dict[item]
                    if (new_property_arr>=0).all() == True:
                        temp = set([item])
                        for subitem in reversed_fork_list_dict[item]:
                            temp.update(cur_layer_list[subitem])
                        cur_layer_list[item] = temp
                        # print(cur_layer_list)
                        
                        
                        temp_property_arr = np.minimum(temp_property_arr,new_property_arr)
                        alloc_dict[Num].append(item)
                        have_been_alloc_list[item] = Num
                        
                        temp_all_nodes_list,temp_extra_node_list,temp_reversed_fork_node_list = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                        all_nodes_list += temp_all_nodes_list
                        extra_nodes_list += temp_extra_node_list
                        ## 检查reverse_fork节点后的节点
                        for subitem in temp_all_nodes_list:
                            reversed_fork_list_dict[subitem].add(item)
                        
                        for subitem in temp_reversed_fork_node_list:
                            if subitem in reversed_fork_node_dict:
                                reversed_fork_node_dict[subitem] = np.minimum(temp_property_arr,reversed_fork_node_dict[subitem])  
                                reversed_fork_list_dict[subitem].add(item)
                            else:
                                reversed_fork_node_dict[subitem] = temp_property_arr
                                reversed_fork_list_dict[subitem].add(item)
                    else:
                        temp_property_arr = temp_property_arr + property_dict[item]
                        
                else:
                    temp = set([item])
                    for subitem in reversed_fork_list_dict[item]:
                        temp.update(cur_layer_list[subitem])
                    cur_layer_list[item] = temp
                    # print(cur_layer_list)
                    ## 单路节点
                    alloc_dict[Num].append(item)
                    have_been_alloc_list[item] = Num
                    # added_point.append(item)
                    temp_all_nodes_list,temp_extra_node_list,temp_reversed_fork_node_list = find_correct_nodes(cob_G,item,have_been_alloc_list,Num)
                    # print(item)
                    # print(temp_extra_node_list)
                    all_nodes_list += temp_all_nodes_list
                    extra_nodes_list += temp_extra_node_list
                    
                    for subitem in temp_all_nodes_list:
                        reversed_fork_list_dict[subitem].add(item)
                    
                    for subitem in temp_reversed_fork_node_list:
                        if subitem in reversed_fork_node_dict:
                                reversed_fork_node_dict[subitem] = np.minimum(temp_property_arr,reversed_fork_node_dict[subitem])  
                                reversed_fork_list_dict[subitem].add(item)
                        else:
                            reversed_fork_node_dict[subitem] = temp_property_arr
                            reversed_fork_list_dict[subitem].add(item)
        # print(temp_all_nodes_list)
        if temp_all_nodes_list != []:
            temp_temp_all_nodes_list,temp_temp_extra_node_list,temp_property_arr,reversed_fork_node_dict,reversed_fork_list_dict,cur_layer_list = DP_head_tree_recursive_update(temp_property_arr,temp_all_nodes_list,Num,cob_G,reversed_fork_node_dict,reversed_fork_list_dict,cur_layer_list)
            extra_nodes_list += temp_temp_extra_node_list
            all_nodes_list += temp_temp_all_nodes_list
            # added_point = temp_added_point

    return all_nodes_list,extra_nodes_list,temp_property_arr,reversed_fork_node_dict,reversed_fork_list_dict,cur_layer_list
    

In [266]:
# mini_ = 100
# for i in tqdm(range(5000)):
cur_layer_list = defaultdict(set)
acc_limit_arr = np.zeros((10000,4))
point_list = [start_point]
Num = 0  ## 层级
TCM_even = 0  ## 偶数数量
alloc_dict = defaultdict(list)
have_been_alloc_list = {}
TCM_even_list = []
all_len = 0
while True:
    extra_point_list = []
    if  16 <= Num and Num<= 31:
        temp_property_arr =np.array([1,3,56+acc_limit_arr[Num-16,2],64+acc_limit_arr[Num-16,3]]) - acc_limit_arr[Num-16,:]
        orig_arr = temp_property_arr
    else:
        temp_property_arr = np.array([1,2,56,64])
        orig_arr = temp_property_arr
    # if Num==29:
    #     print(temp_property_arr)
    # print(temp_property_arr)
    # print(point_list)
    # print(have_been_alloc_list)
    temp_all_nodes_list,temp_extra_node_list,temp_property_arr,reversed_fork_node_dict,reversed_fork_list_dict,cur_layer_list =DP_head_tree_recursive_update(temp_property_arr,point_list,Num,cob_G,{})
   
    
    
    break
    # print(point_list)
    # print(temp_all_nodes_list)
    point_list =  temp_all_nodes_list + temp_extra_node_list + point_list
    # print(reversed_fork_node_dict)
    # print(point_list)
    for item in added_point:
        point_list.remove(item)
    acc_limit_arr[Num,:] =  orig_arr-temp_property_arr
    
    
    if (Num % 2) == 0 and temp_property_arr[0]== 0 and Num-16 not in TCM_even_list:
        TCM_even += 1
        TCM_even_list.append(Num)
    # print(f'{Num} 级共有 {len(alloc_dict[Num])} 个block')
    all_len = all_len + len(alloc_dict[Num])
    Num += 1
    
    if point_list == []:
        break
        # if Num==29:
        #     break
    # if Num<mini_:
    #     mini_ = Num
    #     print(mini_)
    #     df5 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in alloc_dict.items() ]))
    #     df5 = df5.transpose()
    #     # df4.drop(index=0)
    #     df5.to_csv('second_problem_w_greedy_tree.csv', header=False)
    #     pd.DataFrame(acc_limit_arr).to_csv("first_problem_w_greedy_alloc.csv", header=False)
# break

In [267]:
dp = np.zeros((len(cur_layer_list)+1,temp_property_arr[0]+1,temp_property_arr[1]+1,temp_property_arr[2]+1,temp_property_arr[3]+1))

In [294]:
property_cal_dict = defaultdict(lambda: np.zeros(4,dtype=int))

In [295]:
for item in cur_layer_list:
    for subitem in cur_layer_list[item]:
        property_cal_dict[item] += property_dict[subitem]

In [297]:
index_map_ele = {}
cal = 1
for item in cur_layer_list:
    index_map_ele[cal] = item
    cal+=1

In [298]:
dp_dict = defaultdict(set)

In [299]:
for item in index_map_ele:
    if (property_cal_dict[index_map_ele[item]]<=np.array([0,0,0,0])).all() == True:
        dp[(item,0,0,0,0)] = len(cur_layer_list[index_map_ele[item]])
        dp_dict[(item,0,0,0,0)] = deepcopy(cur_layer_list[index_map_ele[item]])

In [300]:
for i in tqdm(range(1,len(cur_layer_list)+1)):
    for j in range(1,temp_property_arr[0]+1):
        for k in range(1,temp_property_arr[1]+1):
            for l in range(1,temp_property_arr[2]+1):
                for m in range(1,temp_property_arr[3]+1):
                    dp[i,j,k,l,m] = dp[i-1,j,k,l,m]
                    dp_dict[(i,j,k,l,m)] = dp_dict[(i-1,j,k,l,m)]
                    if (np.array([j,k,l,m])>=property_cal_dict[index_map_ele[i]]).all() == True:
                        check_point = dp_dict[(i-1,j-property_cal_dict[index_map_ele[i]][0],k-property_cal_dict[index_map_ele[i]][1],l-property_cal_dict[index_map_ele[i]][2],m-property_cal_dict[index_map_ele[i]][3])]
                        if len(check_point & cur_layer_list[index_map_ele[i]])==0:
                            dp[i,j,k,l,m] = max(dp[i,j,k,l,m],dp[i-1,j-property_cal_dict[index_map_ele[i]][0],k-property_cal_dict[index_map_ele[i]][1],l-property_cal_dict[index_map_ele[i]][2],m-property_cal_dict[index_map_ele[i]][3]]+len(cur_layer_list[index_map_ele[i]]))
                            dp_dict[(i,j,k,l,m)] = dp_dict[(i-1,j-property_cal_dict[index_map_ele[i]][0],k-property_cal_dict[index_map_ele[i]][1],l-property_cal_dict[index_map_ele[i]][2],m-property_cal_dict[index_map_ele[i]][3])] | cur_layer_list[index_map_ele[i]]

100%|██████████| 34/34 [00:03<00:00,  8.72it/s]
